# AIoT Energy Forecasting Tutorial

In [ ]:
import json
with open('parameters.json') as f:
    params = json.load(f)
params

In [ ]:
# Bottom-Up Model
N = params['device']['N_j0']
P = params['device']['P_j']
delta_t = params['device']['delta_t']
E_hw = N * P * delta_t / 1e12
print(f"Bottom-Up Energy: {E_hw:.2f} TWh/year")

In [ ]:
# Top-Down Model
import numpy as np
N0 = params['workload']['N0']
e0 = params['workload']['e0']
rho = params['workload']['rho']
rN = params['workload']['r_N']
K = params['workload']['K']
t0 = 0
E_exp0 = N0 * e0 * np.exp((rN - rho) * t0) / 1e12
A = (K / N0) - 1
E_log0 = (K * e0 * np.exp(-rho * t0) / (1 + A * np.exp(-rN * t0))) / 1e12
print(f"Exponential Estimate: {E_exp0:.2f} TWh/year")
print(f"Logistic Estimate: {E_log0:.2f} TWh/year")

In [ ]:
# Growth Forecast Plot
import matplotlib.pyplot as plt
E0 = params['growth']['E0']
alpha = params['growth']['alpha']
Emax = params['growth']['Emax']
beta = params['growth']['beta']
t = np.linspace(0, 10, 100)
E_exp = E0 * np.exp((alpha - rho) * t)
E_log = Emax / (1 + ((Emax / E0 - 1) * np.exp(-beta * t)))
plt.figure(figsize=(8,5))
plt.plot(2024 + t, E_exp, '--', label='Exponential')
plt.plot(2024 + t, E_log, '-', label='Logistic')
plt.title("AI Energy Forecasts")
plt.xlabel("Year")
plt.ylabel("Energy (TWh/year)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Simple ABM
num_devices = 1000
timesteps = 50
P_idle = 1
P_peak = 10
energy = []
state = np.zeros(num_devices)
for t in range(timesteps):
    state = np.random.rand(num_devices) < 0.2
    power = P_idle + (P_peak - P_idle) * state
    energy.append(np.sum(power) / 1e3)
plt.figure(figsize=(8,5))
plt.plot(range(timesteps), energy)
plt.title("ABM Energy Evolution")
plt.xlabel("Hour")
plt.ylabel("Energy (MWh)")
plt.grid(True)
plt.show()

In [ ]:
# Interactive Forecasting
import ipywidgets as widgets
from IPython.display import display

def plot_energy(alpha, beta, rho):
    t = np.linspace(0, 10, 100)
    E0 = 450
    Emax = 1500
    E_exp = E0 * np.exp((alpha - rho) * t)
    E_log = Emax / (1 + ((Emax / E0 - 1) * np.exp(-beta * t)))
    plt.figure(figsize=(8,5))
    plt.plot(2024 + t, E_exp, '--', label='Exponential')
    plt.plot(2024 + t, E_log, '-', label='Logistic')
    plt.title("Interactive Forecast")
    plt.xlabel("Year")
    plt.ylabel("Energy (TWh/year)")
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(plot_energy,
    alpha=widgets.FloatSlider(value=0.15, min=0.05, max=0.3, step=0.01),
    beta=widgets.FloatSlider(value=0.45, min=0.1, max=1.0, step=0.05),
    rho=widgets.FloatSlider(value=0.30, min=0.1, max=0.5, step=0.01))

In [ ]:
# Export forecast to CSV
import pandas as pd
t = np.linspace(0, 10, 100)
years = 2024 + t
E_exp = E0 * np.exp((alpha - rho) * t)
E_log = Emax / (1 + ((Emax / E0 - 1) * np.exp(-beta * t)))
df = pd.DataFrame({'Year': years, 'Exponential_Forecast': E_exp, 'Logistic_Forecast': E_log})
df.to_csv('forecast_output.csv', index=False)
df.head()